In [1]:
import os
import subprocess

os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/usr/bin/python3.9'

# Set HADOOP_CONF_DIR environment variable
os.environ['HADOOP_CONF_DIR'] = '/usr/odp/1.2.4.0-102/hadoop/etc/hadoop'

# Set ARROW_LIBHDFS_DIR environment variable
os.environ['ARROW_LIBHDFS_DIR'] = '/usr/odp/1.2.4.0-102/hadoop/lib/native/'

# Set CLASSPATH enviroment variable
classpath = subprocess.check_output(['hadoop', 'classpath', '--glob'])
os.environ['CLASSPATH'] = classpath.decode('utf-8')

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import *

import numpy as np
import pandas as pd

In [2]:
spark = (
    SparkSession.builder.appName("kedro_dev")
        .config("spark.master", "yarn")
        .config("spark.deploy.mode", "cluster")
        .config("spark.driver.cores", "1")
        .config("spark.driver.memory", "1g")
        .config("spark.executor.cores", "1")
        .config("spark.executor.memory", "1g")
        .config("spark.executor.instances", "2")
        .config("spark.executor.memoryOverhead", "512m")
        .config("spark.sql.legacy.timeParserPolicy", "LEGACY")
        .config("spark.sql.parquet.enableVectorizedReader", "false")
        .config("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
        .config("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
        .config("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
        .config("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")
        .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/20 14:02:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/20 14:02:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/06/20 14:02:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/06/20 14:02:56 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/06/20 14:02:56 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/06/20 14:02:56 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/06/20 14:02:57 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
25/06/20 14:02:57 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back 

In [28]:
df = pd.read_csv('/home/VT_TTDLPT_MINHPN5/kedro-dev/data/01_raw/raw.csv')

# Hiển thị 5 dòng đầu tiên
spark_df = spark.createDataFrame(df)

spark_df.write \
    .mode("overwrite") \
    .parquet("hdfs://10.53.2.40:8020/user/VT_TTDLPT_MINHPN5/test/raw/2025-01-07")

In [ ]:
df_1 = pd.read_csv('/home/VT_TTDLPT_MINHPN5/kedro-dev/data/01_raw/intermediate_information.csv')

# Hiển thị 5 dòng đầu tiên
spark_df_1 = spark.createDataFrame(df_1)

spark_df_1.write \
    .mode("overwrite") \
    .partitionBy("processed_date") \
    .parquet("hdfs://10.53.2.40:8020/user/VT_TTDLPT_MINHPN5/test/intermediate_information")


In [4]:
spark_df_reloaded = spark.read.parquet(
    "hdfs://10.53.2.40:8020/user/VT_TTDLPT_MINHPN5/test/feature_information"
)

spark_df_reloaded.show()

+---+--------------------+--------------+
| id|                name|processed_date|
+---+--------------------+--------------+
|  6|MinhfLKFNACN JDAS...|    2025-01-06|
|  6|MinhfLKFNACN JDAS...|    2025-01-06|
|  7|aslkjBFNJA DNJHVS...|    2025-01-07|
|  7|aslkjBFNJA DNJHVS...|    2025-01-07|
+---+--------------------+--------------+



In [6]:
spark.stop()

In [29]:
spark_df_reloaded = spark.read.parquet(
    "hdfs://10.53.2.40:8020/user/VT_TTDLPT_MINHPN5/test/raw/{2025-01-06,2025-01-07}"
)

In [30]:
spark_df_reloaded.show()

+---+--------------------+--------------+
| id|                name|processed_date|
+---+--------------------+--------------+
|  7|ssdvbskadbvkasahd...|    2025-01-07|
|  6|sasahdadasjbdasjdhdv|    2025-01-06|
+---+--------------------+--------------+

